# Task2: 基于PyTorch框架的手写数字识别
## 引入相关库

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import os

## 设置超参数

In [9]:
learning_rate = 0.01
epoches = 100
batch_size = 128

## Re-implement MLP
利用PyTorch的内置神经网络模块（torch.nn.Module的子类），在MLP类中实现两个函数：
+ 在__init__函数中，定义一个网络结构为[784-245-128-10]的MLP模型结构
+ 在forward函数中，实现该MLP模型的前向传播过程

下面是一些供你参考/可能用到的API函数：

- torch.nn.Linear(*in_features*, *out_features*, *bias=True*, *device=None*, *dtype=None*) [
  Link](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
  - in_features: 输入网络层的特征维度
  - out_features: 输出网络层的特征维度
- torch.nn.Module.forward(**input*) [Link](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.forward)
  - 执行模型的前向过程，继承nn.Module类的类实例可以直接通过变量名加括号实现forward函数的调用，不需要写明调用forward函数
  - 如定义了MLP(nn.Module)，则对于mlp = MLP()，可以通过mlp(**input*)调用
- torch.Tensor.reshape(*shape*) [Link](https://pytorch.org/docs/stable/generated/torch.Tensor.reshape.html)
  - shape: 当前tensor希望修改为的形状，如(2, 2)或(-1, 3)
    - -1指该维度大小根据原数据维度大小和其它给定维度大小计算得到，至多可以给一个-1
- torch.nn.Sigmoid() [Link](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html)

In [10]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # TODO: 定义上文要求的MLP模型结构
        self.fc1 = nn.Linear(784, 245)
        self.fc2 = nn.Linear(245, 128)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        # TODO: 定义MLP模型的前向过程
        xL = x.view(-1, 784)
        h1 = self.fc1(xL)
        h2 = self.fc2(h1)
        o = self.fc3(h2)
        return o

## 示例化MLP

In [11]:
mlp = MLP()

## 定义损失函数、优化算法

- torch.nn.CrossEntropyLoss(*weight=None*, *size_average=None*, *ignore_index=- 100*, *reduce=None*, *reduction='mean'*, *label_smoothing=0.0*) [Link](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
  - loss.backward(): loss通过特定的计算方式获得，如调用CrossEntropyLoss；对loss执行backward()会为计算图中涉及的tensor反向计算梯度，累积到tensor.grad上
- torch.optim.SGD(*params*, *lr=<required parameter>*, *momentum=0*, *dampening=0*, *weight_decay=0*, *nesterov=False*, ***, *maximize=False*, *foreach=None*, *differentiable=False*)  [Link](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)
  - params: 需优化的参数Tensor
  - lr: 参数优化的学习率
  - zero_grad(): 清空相关参数上累积的梯度
  - step(): 根据tensor上累积的梯度，进行一次参数更新

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

## 加载数据集

- 自动下载MNIST数据集到./MNIST路径

In [13]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
transform = transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root="./MNIST", train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root="./MNIST", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, drop_last=True)

## 训练模型

此处关于loss和optimizer的用法请参考上一段落的API介绍。

In [14]:
mlp.train()

for e in range(epoches):
    t = tqdm(train_loader)
    for img, label in t:
        # Forward img and compute loss
        pred = mlp(img)
        loss = criterion(pred, label)
        
        # TODO: 基于优化器的使用方法，完成反向梯度传播、参数更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        t.set_postfix(epoch=e, train_loss=loss.item())

100%|██████████| 468/468 [00:07<00:00, 64.20it/s, epoch=99, train_loss=0.514] 


## 测试模型

- torch.argmax(*input*, *dim*, *keepdim=False*) [Link](https://pytorch.org/docs/stable/generated/torch.argmax.html)
  - input: 计算基于的tensor
  - dim: 希望按哪个维度求max下标

In [15]:
mlp.eval()

correct_cnt, sample_cnt = 0, 0

t = tqdm(test_loader)
for img, label in t:
    # Predict label for img
    img = img.reshape(img.shape[0], -1)
    pred = mlp(img)
    pred_label = pred.argmax(dim=1)
    
    correct_cnt += (pred_label == label).sum().item()
    sample_cnt += pred_label.shape[0]

    t.set_postfix(test_acc=correct_cnt/sample_cnt)

100%|██████████| 78/78 [00:01<00:00, 76.87it/s, test_acc=0.895]


## 保存模型

- 将完成训练的模型保存到服务器的model/目录下

- ModelScope服务器端无法长久保存文件，因此请及时下载、本地保存你完成的代码，以及模型的参数文件（model/mlp.pt）。

In [16]:
if not os.path.exists('model/'):
    os.mkdir('model/')

torch.save(mlp.state_dict(), 'model/mlpO.pt')